In [1]:
import turicreate as tc
import pandas as pd
import numpy as np
import random

class RecommenderClient:
    def __init__(self,data):
        # data is what is returned by Vishnu's client
        self.data = data;
        
        #the ref data is the raw data table
        ref_data = self.ref_data = pd.DataFrame.from_dict(data)
        self.num_of_choices = len(ref_data);
        
        #construct data frame that contains only the training categories
        model_data = self.model_data = ref_data[['category', 'tags']]
        model_data['index'] = model_data.index
        
        self.categories = self.model_data['category'].unique()
        
        #create turicreate SFrame
        tc_data = self.tc_data = tc.SFrame(model_data)
        
        #create model
        model = self.model = tc.recommender.item_content_recommender.create(tc_data, item_id = 'index', verbose=False)
        
        
    def cold_start(self,k=10):
        #k is the number of items to suggest. 
        #randomly suggests k items
        # returns list of indices. 
        
        suggestion_id = [random.randrange(self.num_of_choices) for _ in range(k)]
        
        return suggestion_id
        
        
    def suggest(self, k = 10, likes = [], dislikes = [], categories = []):
        #likes is the list of items that have been liked. 
        #categories is either a string matching the category, or a list of strings matching the categories
        #returns pandas dataframe with index and the score

        
        #create list of allowed items
        if categories  == []:
            allowed_index = list(range(self.num_of_choices));
        else:
            allowed_index = [i for i in range(self.num_of_choices) if self.model_data['category'][i] in categories]
            #allowed_items = tc.SFrame(allowed_index)
        
        recommendations = self.model.recommend_from_interactions(observed_items = likes,
                                                                 k=k,
                                                                 exclude=None, #dislikes,#tc.SFrame(self.model_data.iloc[dislikes]),
                                                                 items=allowed_index,
                                                                 new_user_data=None,
                                                                 new_item_data=None,
                                                                 exclude_known=True,
                                                                 diversity=0,
                                                                 random_seed=None,
                                                                 verbose=False)
        
        return recommendations

In [2]:
import pickle
import os
os.chdir('../data-import/')
if 1:
    %run main.py 
else:
    data = pickle.load( open( "london_poi.pickle", "rb" ) )
os.chdir('../recommender/')

SyntaxError: invalid syntax (main.py, line 11)

In [3]:
import pickle
import pandas
import os

os.chdir('../data-import/')
from AmadeusClient import AmadeusClient

am = AmadeusClient()

cities = pd.read_csv('defaultCities.csv')



In [4]:
cities

,Name,Latitude,Longitude
0,Paris,48.8566,2.3522
1,Rome,41.9028,12.4964
2,Venice,45.4408,12.3155
3,London,51.5074,-0.1278
4,Barcelona,41.3851,2.1734
5,Florence,43.7696,11.2558
6,Prague,50.0755,14.4378
7,Madrid,40.4168,3.7038
8,Vienna,48.2082,16.3738
9,Istanbul,41.0082,28.9784


In [14]:
F"City: {cities['Name'][0]}"

'City: Paris'

In [11]:
f'City: {cities['Name'][0]}'

SyntaxError: invalid syntax (<ipython-input-11-ca7ded27e0c8>, line 1)

In [22]:
i=0
citydata = am.get_poi(lat=cities['Latitude'][i], lon=cities['Longitude'][i], name=cities['Name'][i],lim=20)

In [23]:
len(citydata)

20

In [30]:
data = []
for i in range(len(cities)):
    citydata = am.get_poi(lat=cities['Latitude'][i], lon=cities['Longitude'][i], name=cities['Name'][i],lim=20)
    print('Loaded city: %s, with %i POIs'%(cities['Name'][i], len(citydata)))
    #f'Len: {len(citydata)}'
    data += citydata

KeyboardInterrupt: 

In [25]:
len(data)

200

In [ ]:

#data = am.get_poi(lat=51.50, lon=-0.177)
data = am.get_poi(lat=41.39715, lon=2.160873)

#pickle.dump(data, open('london_poi.pickle', 'wb'))
pickle.dump(data, open('spain_poi.pickle', 'wb'))



In [ ]:
data

In [ ]:
rc = RecommenderClient(data)

In [ ]:
rc.cold_start()

In [ ]:
suggestions = rc.suggest(k  = 10 , likes = [0,], dislikes=[83,64],categories=['RESTAURANT','SHOPPING'])

In [ ]:
rc.ref_data.iloc[list(suggestions['index'])]

In [ ]:
rc.model_data['category'].unique()

In [ ]:
rc.model_data[(rc.model_data['category'] in ['SIGHTS', 'SHOPPING'])]

In [ ]:
any(rc.model_data['category']) in ['SIGHTS', "SHOPPING"]

In [ ]:
allowed_index = [i for i in range(rc.num_of_choices) if rc.model_data['category'][i] in ['SIGHTS', "SHOPPING"]]

In [ ]:
allowed_index

In [ ]:
rc.model_data['category']

In [ ]:
rc.tc_data

In [ ]:
list(range(4))

In [ ]:
disallowed=[]

In [ ]:
tc.SFrame(rc.model_data.iloc[disallowed])

In [ ]:
rc.model_data.iloc[[5, 6]]

In [ ]:
rc.categories

In [ ]:
cat = 'SIGHTS'

In [ ]:
likes = [0,1,3,4]

In [ ]:
rc.model.recommend_from_interactions(observed_items = likes, new_item_data=tc_spain_data)

In [ ]:
rc.tc_data

In [ ]:
rc.tc_data

In [ ]:
items = tc.SFrame({'category':['SIGHTS', 'SIGHTS', 'SIGHTS'], 'tags': [['sightseeing', 'restaurant', 'cheap'], ['cheap', 'museum'], ['landmark','vegetarian']],'index':[0,1,2]})

In [ ]:
items

In [ ]:
spain_data = pd.DataFrame.from_dict(data)[['category', 'tags']]

In [ ]:
spain_data['index'] = spain_data.index

In [ ]:
tc_spain_data = tc.SFrame(spain_data)

In [ ]:
tc_spain_data